In [6]:
import pandas as pd
from collections import defaultdict
import random


In [2]:
# Cargar el CSV (puedes ajustar el nombre del archivo si es necesario)
df = pd.read_csv("dataset/mentorAvailability.csv")

# Mostrar las primeras filas
df.head()

,MentorID,Slot1,Slot2,Slot3,Slot4,Slot5,Slot6,Slot7,Slot8,Slot9,Slot10
0,M01,1,1,1,1,0,0,0,1,1,1
1,M02,1,1,0,1,0,0,1,0,0,1
2,M03,0,1,0,1,1,0,1,0,0,1
3,M04,0,1,0,0,0,0,1,1,0,1
4,M05,1,1,0,1,1,1,0,0,0,1


In [3]:
# Extraer los IDs de mentores
mentores = df['MentorID'].tolist()

# Extraer los horarios disponibles como listas por mentor
disponibilidades = df.drop(columns='MentorID').values.tolist()

# Crear diccionario mentor -> slots disponibles
mentor_slots = {
    mentor: [i for i, disponible in enumerate(slots) if disponible == 1]
    for mentor, slots in zip(mentores, disponibilidades)
}

mentor_slots


{'M01': [0, 1, 2, 3, 7, 8, 9],
 'M02': [0, 1, 3, 6, 9],
 'M03': [1, 3, 4, 6, 9],
 'M04': [1, 6, 7, 9],
 'M05': [0, 1, 3, 4, 5, 9],
 'M06': [0, 4, 5, 7, 9],
 'M07': [1, 5, 6, 8, 9],
 'M08': [0, 1, 2, 3, 4, 5, 7, 8, 9],
 'M09': [0, 1, 2, 4, 5, 9],
 'M10': [0, 2, 3, 4, 6],
 'M11': [0, 5],
 'M12': [0, 1, 9],
 'M13': [0, 1, 2, 3, 4, 5, 7, 8, 9],
 'M14': [2, 4, 6, 7, 8, 9],
 'M15': [1, 2, 4, 5, 9],
 'M16': [0, 1, 4, 6, 7],
 'M17': [1, 2, 5, 8],
 'M18': [0, 4, 5, 6, 7, 9],
 'M19': [2, 7, 8],
 'M20': [4, 6, 8]}

In [5]:
def calcular_choques(asignacion):
    horario_ocupado = defaultdict(int)
    for slot_dupla in asignacion.values():
        for slot in slot_dupla:
            horario_ocupado[slot] += 1
    return sum(v - 1 for v in horario_ocupado.values() if v > 1)


In [7]:
def generar_vecino(asignacion_actual, mentor_slots):
    nuevo = asignacion_actual.copy()
    mentor = random.choice(list(mentor_slots.keys()))
    opciones = [
        (i, j) for i in mentor_slots[mentor]
                for j in mentor_slots[mentor]
                if i < j and j - i == 1
    ]
    if not opciones:
        return nuevo
    nuevo[mentor] = random.choice(opciones)
    return nuevo


In [8]:
def busqueda_local(mentor_slots, max_iter=1000):
    # Inicialización: elegir una franja de 2h contigua por mentor
    asignacion = {}
    for mentor, slots in mentor_slots.items():
        candidatos = [(i, j) for i in slots for j in slots if i < j and j - i == 1]
        if candidatos:
            asignacion[mentor] = random.choice(candidatos)
        else:
            asignacion[mentor] = random.sample(slots, 2) if len(slots) >= 2 else slots

    mejor = asignacion
    mejor_costo = calcular_choques(mejor)

    for _ in range(max_iter):
        vecino = generar_vecino(mejor, mentor_slots)
        costo = calcular_choques(vecino)

        if costo < mejor_costo:
            mejor = vecino
            mejor_costo = costo

        if mejor_costo == 0:
            break

    return mejor, mejor_costo


In [9]:
solucion, choques = busqueda_local(mentor_slots)

print("Asignación final:")
for mentor, slots in solucion.items():
    print(f"{mentor}: {slots}")

print(f"Choques: {choques}")


Asignación final:
M01: (8, 9)
M02: (0, 1)
M03: (3, 4)
M04: (6, 7)
M05: (4, 5)
M06: (4, 5)
M07: (5, 6)
M08: (3, 4)
M09: (4, 5)
M10: (2, 3)
M11: [5, 0]
M12: (0, 1)
M13: (0, 1)
M14: (8, 9)
M15: (4, 5)
M16: (6, 7)
M17: (1, 2)
M18: (4, 5)
M19: (7, 8)
M20: [4, 8]
Choques: 30
